In [ ]:
import datetime
import lxml.etree as et
import os

from article_class import Article
from samples.corpus_analysis import get_random_list_of_dois

# Main directory of article XML files
corpusdir = 'allofplos_xml'

# Temp folder for downloading and processing n/'ew articles
newarticledir = 'new_plos_articles'

# URL bases for PLOS's raw article XML
EXT_URL_TMP = 'http://journals.plos.org/plosone/article/file?id={0}&type=manuscript'
INT_URL_TMP = 'http://contentrepo.plos.org:8002/v1/objects/mogilefs-prod-repo?key={0}.XML'
URL_TMP = EXT_URL_TMP


In [ ]:
import csv
dois = get_random_list_of_dois(count=500)
article_array = []
for doi in dois:
    article = Article(doi)
    article_data = []
    if article.type_ not in ('correction', 'retraction') and article.pubdate > datetime.datetime(2017, 1, 1):
#         if type(article.author) == dict:
#             mult_authors = False
#             article_data = (mult_authors, article.author['last'], article.author['first'], article.author['email'], article.title, article.journal, article.type_, article.doi, article.page, article.get_pubdate(string_=True))
#             article_array.append(article_data)
#             pass
#         elif type(article.author) == list:
#             mult_authors = True
#             for author in article.author:
#                 article_data = (mult_authors, author['last'], author['first'], author['email'], article.title, article.journal, article.type_, article.doi, article.page, article.get_pubdate(string_=True))
#                 print(author['first'], author['last'], author['email'], article.title)
#                 article_array.append(article_data)
#         else:
#             print('Error reading author list: {}'.format(article.doi))
        print(article.doi, article.author)

In [ ]:
with open('testing_author_dupes2.csv', 'w', encoding='utf-8') as out:
        csv_out = csv.writer(out)
        csv_out.writerow(['mult. authors', 'last name', 'first name', 'email', 'title', 'journal', 'type', 'doi', 'link', 'pubdate'])
        for row in article_array:
            csv_out.writerow(row)

In [ ]:
article = Article('10.1371/journal.pone.0187065')
foo = article.author
# dois = get_random_list_of_dois(count=100)
# for doi in dois:
#     article = Article(doi)
#     for author in article.author:
#         print(author)

In [ ]:
article = Article('10.1371/journal.pmed.0020043')
article.author

In [ ]:
article = Article('10.1371/journal.pone.0182368')
article.author

In [ ]:
article = Article('10.1371/journal.pone.0170783')
article.author